In [1]:
!pip install kagglehub pandas scikit-learn streamlit numpy

   ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
   ---------------------------- ----------- 30.7/42.9 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 42.9/42.9 kB 693.9 kB/s eta 0:00:00


In [2]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
import streamlit as st

In [3]:
# Download dataset
path = kagglehub.dataset_download("kazanova/sentiment140")
print("Path to dataset files:", path)

# Load the dataset
data = pd.read_csv(f"{path}/training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1', header=None)
data.columns = ['target', 'id', 'date', 'flag', 'user', 'text']

# Keep only target and text columns
data = data[['target', 'text']]
# Convert target to binary: 0 for negative, 1 for positive
data['target'] = data['target'].apply(lambda x: 1 if x == 4 else 0)

# Display dataset information
print(data.head())
print(data['target'].value_counts())

100%|██████████| 80.9M/80.9M [00:11<00:00, 7.35MB/s]

Extracting files...


Path to dataset files: C:\Users\Rohan\.cache\kagglehub\datasets\kazanova\sentiment140\versions\2
   target                                               text
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  is upset that he can't update his Facebook by ...
2       0  @Kenichan I dived many times for the ball. Man...
3       0    my whole body feels itchy and like its on fire 
4       0  @nationwideclass no, it's not behaving at all....
target
0    800000
1    800000
Name: count, dtype: int64


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Function to clean text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)     # Remove mentions
    text = re.sub(r'#\w+', '', text)     # Remove hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.lower().strip()

# Clean the text data
data['text'] = data['text'].apply(clean_text)

# Split data
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2, random_state=42)

# Feature extraction
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [6]:
# Train the model
model = LogisticRegression(max_iter=200)
model.fit(X_train_vec, y_train)

# Evaluate model
y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print("Model accuracy:", accuracy)

Model accuracy: 0.790371875


C:\Users\Rohan\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
# Save model and vectorizer
joblib.dump(model, 'sentiment_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']